<a href="https://colab.research.google.com/github/c-c-c-c/kaggle_titanic/blob/master/myWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#追記
import json
import datetime
import math

#importじゃないけど、個人的には省略されて良いことがあまりないので・・
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/test.csv')
gender_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/data/gender_submission.csv')
# gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [40]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [0]:
train["is_train"] = 1
test["is_train"]  = 0

In [0]:
df_merged = pd.concat([train,test])

In [0]:
# df_merged["Embarked"] = df_merged["Embarked"].fillna("NNN")

In [0]:
# # df_merged[ df_merged["Embarked"]==999]

# # df_merged[df_merged["Sex"].isnull()]

# # df_merged

# le = preprocessing.LabelEncoder()
# le.fit(df_merged["Embarked"])
# df_merged["Embarked"] = le.transform(df_merged["Embarked"])


In [0]:

# df_merged[df_merged["Embarked"].isnull()]
# set(df_merged["Embarked"].values)

In [0]:
# df_merged[df_merged["Sex"].isnull()]
# set(df_merged["Sex"].values)

In [0]:
# for column in ['Sex','Embarked']:
for column in ['Sex','Embarked']:
#     print(titanic[column].shape)
    df_merged[column] = df_merged[column].fillna("NNN")
    le = preprocessing.LabelEncoder()
    le.fit(df_merged[column])
    df_merged[column] = le.transform(df_merged[column])




In [0]:
train = df_merged[df_merged["is_train"] ==1]

In [0]:
test = df_merged[df_merged["is_train"] ==0]

In [0]:
# # SexとEmbarkedのlabel encoding
# train = pd.get_dummies(train, columns=['Sex', 'Embarked']) 
# test = pd.get_dummies(test, columns=['Sex', 'Embarked'])
# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [0]:
feat_cols =  list(train.columns)
feat_cols.remove('Survived')
feat_cols.remove('is_train')

categoricals = ["Pclass", 'SibSp','Parch','Sex','Embarked']

In [0]:
# feat_cols

In [0]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            # 'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'num_leaves': 100,
            'min_data_in_leaf':24,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

In [0]:
folds = 5
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

In [54]:
models = []
for train_index, val_index in kf.split(train, train['Survived']):
    train_X = train[feat_cols].iloc[train_index]
    val_X = train[feat_cols].iloc[val_index]
    train_y = train["Survived"].iloc[train_index]
    val_y = train["Survived"].iloc[val_index]

    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.472191	valid_1's binary_logloss: 0.524273
[200]	training's binary_logloss: 0.411625	valid_1's binary_logloss: 0.486109
[300]	training's binary_logloss: 0.382243	valid_1's binary_logloss: 0.472528
[400]	training's binary_logloss: 0.363404	valid_1's binary_logloss: 0.463965
[500]	training's binary_logloss: 0.348365	valid_1's binary_logloss: 0.459558
[600]	training's binary_logloss: 0.336182	valid_1's binary_logloss: 0.454063
[700]	training's binary_logloss: 0.326615	valid_1's binary_logloss: 0.45009
[800]	training's binary_logloss: 0.317845	valid_1's binary_logloss: 0.445997
[900]	training's binary_logloss: 0.310041	valid_1's binary_logloss: 0.44271
[1000]	training's binary_logloss: 0.302881	valid_1's binary_logloss: 0.441255
[1100]	training's binary_logloss: 0.296869	valid_1's binary_logloss: 0.440371
[1200]	training's binary_logloss: 0.291201	valid_1's binary_logloss: 0.437599
[1300]	train

In [0]:
predicts_sum = []
for i, tmp_model in enumerate(models):
    predicts = tmp_model.predict(test[feat_cols])

    if  i == 0 :
        predicts_sum = predicts
    else:
        for i in range(len(predicts_sum)):
            predicts_sum[i] += predicts[i]

u_predicts = [ x/len(models) for x in predicts_sum ]

In [0]:
test["Survived"] = [ 1 if x > 0.5 else 0 for  x in u_predicts]

# [式 for 任意の変数名 in イテラブルオブジェクト

In [0]:
df_sub = gender_submission.copy()
df_sub["Survived"] = test["Survived"]

In [0]:
df_sub.to_csv('/content/drive/My Drive/Colab Notebooks/kaggle_titanic/titanic/sub.csv',index=False)